In [ ]:
from concurrent.futures import ThreadPoolExecutor
from matplotlib import pyplot as plt
import numpy as np
import os
from pathlib import Path
import torch
from tqdm.notebook import tqdm

from functions.processing.retrieval import loadPT
base_directory = r'F:\\Persönliches\\Git\BioOTon\\XenoCanto'

In [ ]:
def calculate_avg_snr(clean_path, noisy_path):
    diffs = []

    clean_file_paths = list(clean_path.glob("*.pt"))
    noisy_file_paths = list(noisy_path.glob("*.pt"))

    for clean_path, noisy_path in zip(clean_file_paths, noisy_file_paths):
    # Ensure tensors are floats
        print(f"Processing {clean_path} and {noisy_path}")
        clean = loadPT(clean_path)
        clean = clean.float()

        noisy = loadPT(noisy_path)
        noisy = noisy.float()
        
        # Noise is the difference between the two
        noise = noisy - clean
        
        # Calculate power (mean of squares)
        signal_power = torch.mean(abs(clean) ** 2, dim=list(range(1, clean.ndim)))
        noise_power = torch.mean(abs(noise) ** 2, dim=list(range(1, noise.ndim)))
        
        # Calculate SNR in dB
        # Adding a small epsilon to avoid division by zero or log(0)
        snr = 10 * torch.log10(signal_power / (noise_power + 1e-10))

        diffs.append(torch.mean(snr).item())
    
    return diffs, np.mean(diffs)

In [ ]:
def calculate_avg_snr_mt(zipped_paths):
    clean_path, noisy_path = zipped_paths

    # Ensure tensors are floats
    print(f"Processing {clean_path} and {noisy_path}")
    clean = loadPT(clean_path)
    clean = clean.float()

    noisy = loadPT(noisy_path)
    noisy = noisy.float()
    
    # Noise is the difference between the two
    noise = noisy - clean
    
    # Calculate power (mean of squares)
    signal_power = torch.mean(abs(clean) ** 2, dim=list(range(1, clean.ndim)))
    noise_power = torch.mean(abs(noise) ** 2, dim=list(range(1, noise.ndim)))
    
    # Calculate SNR in dB
    # Adding a small epsilon to avoid division by zero or log(0)
    snr = 10 * torch.log10(signal_power / (noise_power + 1e-10))

    return torch.mean(snr).item()

In [ ]:
clean_path = Path(f"./AudioTensors")
noisy_path = Path(f"./AudioTensors_denoised")

clean_file_paths = list(clean_path.glob("*.pt"))
noisy_file_paths = list(noisy_path.glob("*.pt"))

file_paths = zip(clean_file_paths, noisy_file_paths)

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    # Use a list to hold the results so tqdm can track progress
    results = list(tqdm(
        executor.map(lambda f: calculate_avg_snr_mt(f), file_paths), 
        total=len(clean_file_paths),
        desc="Computing signal to noise ratio."
    ))

In [ ]:
# torch.save(results, "SNR.pt")

In [ ]:
plt.plot(results)
plt.title("Avg. SNR among all instances in the Dawn Chorus data")

In [ ]:
mean_snr = np.nanmean(results)
print(f"Mean SNR: {mean_snr}")
mean_squared_snr = np.mean(np.square(results))
print(f"Mean squared SNR: {mean_squared_snr}")
print("###################################\n")

var_snr = np.var(results)
print(f"SNR Variance: {var_snr}")
std_snr = np.std(results)
print(f"SNR Std: {std_snr}")
print("###################################\n")

min_snr = np.min(results)
print(f"Min SNR: {min_snr}")
max_snr = np.max(results)
print(f"Max SNR: {max_snr}")
print("###################################\n")

In [ ]:
clean_path = Path(f"{base_directory}+Tensors")
noisy_path = Path(base_directory+"_denoised")

clean_file_paths = list(clean_path.glob("*.pt"))
noisy_file_paths = list(noisy_path.glob("*.pt"))

file_paths = zip(clean_file_paths, noisy_file_paths)

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    # Use a list to hold the results so tqdm can track progress
    results = list(tqdm(
        executor.map(lambda f: calculate_avg_snr_mt(f), file_paths), 
        total=len(clean_file_paths),
        desc="Computing signal to noise ratio."
    ))

In [ ]:
plt.plot(results)
plt.title("Avg. SNR among all instances in the Xeno-Canto data")

In [ ]:
mean_snr = np.nanmean(results)
print(f"Mean SNR: {mean_snr}")
mean_squared_snr = np.mean(np.square(results))
print(f"Mean squared SNR: {mean_squared_snr}")
print("###################################\n")

var_snr = np.var(results)
print(f"SNR Variance: {var_snr}")
std_snr = np.std(results)
print(f"SNR Std: {std_snr}")
print("###################################\n")

min_snr = np.min(results)
print(f"Min SNR: {min_snr}")
max_snr = np.max(results)
print(f"Max SNR: {max_snr}")
print("###################################\n")